In [5]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### 1. Train Model

#### 1.1. Describe data and split dataset

In [6]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [10]:
# df = describe_dataset("./train.csv")
df = describe_dataset("cleaned_train.csv")
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "H", "label"] = 1
df.loc[df["label"] == "L", "label"] = 2
df.tail(3)

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
28583,0,0.279320,0.659774,-0.057247,0.999669,0.347388,0.625295,-0.291926,0.999771,0.366641,...,0.331177,0.775523,0.866891,0.861069,-0.105977,0.936999,0.860829,0.853743,0.241477,0.750864
28584,0,0.278973,0.659655,-0.056122,0.999653,0.347229,0.624985,-0.291974,0.999774,0.366518,...,0.331179,0.776471,0.866143,0.860887,-0.106509,0.938309,0.860260,0.853562,0.240993,0.753471
28585,0,0.278932,0.659352,-0.048404,0.999686,0.346968,0.623998,-0.287728,0.999794,0.366728,...,0.326128,0.776392,0.866164,0.861528,-0.112991,0.938339,0.860247,0.853922,0.235636,0.753403


In [11]:
# Extract features and class
X = df.drop("label", axis=1)
y = df["label"].astype("int")

In [ ]:
sc = StandardScaler()
# X = pd.DataFrame(sc.fit_transform(X))
X = pd.DataFrame(sc.fit_transform(X), columns=X.columns, index=X.index)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_test.head(3)

7699     2
14138    2
15147    1
Name: label, dtype: int32

#### 1.2. Train model using Scikit-Learn and train set evaluation

In [14]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []

for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=[0, 1, 2])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=[0, 1, 2])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=[0, 1, 2])
    cm = confusion_matrix(y_test, model_results, labels=[0, 1, 2])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))


In [15]:
# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)

pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,KNN,"[0.999, 1.0, 1.0]",0.999825,"[1.0, 0.999, 1.0]","[1.0, 1.0, 1.0]","[[1955, 0, 0], [1, 1829, 0], [0, 0, 1933]]"
1,LR,"[0.999, 1.0, 0.999]",0.999475,"[0.999, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1954, 0, 1], [0, 1830, 0], [2, 0, 1931]]"
2,SVC,"[0.998, 1.0, 0.999]",0.999300,"[0.999, 1.0, 0.998]","[0.999, 1.0, 0.999]","[[1954, 0, 1], [0, 1830, 0], [3, 0, 1930]]"
3,SGDC,"[0.999, 0.999, 0.998]",0.999126,"[0.998, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1951, 1, 3], [0, 1830, 0], [1, 0, 1932]]"
4,RF,"[0.999, 1.0, 1.0]",0.999650,"[1.0, 1.0, 0.999]","[0.999, 1.0, 0.999]","[[1955, 0, 0], [0, 1830, 0], [2, 0, 1931]]"
5,DTC,"[0.995, 1.0, 0.998]",0.997726,"[0.998, 0.999, 0.996]","[0.997, 0.999, 0.997]","[[1952, 0, 3], [1, 1828, 1], [8, 0, 1925]]"
6,NB,"[0.8, 0.921, 0.942]",0.881952,"[0.876, 0.96, 0.814]","[0.836, 0.94, 0.873]","[[1713, 145, 97], [74, 1756, 0], [354, 5, 1574]]"


#### 1.3. Test set evaluation

In [16]:
# test_df = describe_dataset("./test.csv")
test_df = describe_dataset("test.csv")
test_df = test_df.sample(frac=1).reset_index(drop=True)

test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "H", "label"] = 1
test_df.loc[test_df["label"] == "L", "label"] = 2

test_x = test_df.drop("label", axis=1)
test_y = test_df["label"].astype("int")

test_x = pd.DataFrame(sc.transform(test_x))

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

In [ ]:
# Evaluate models on the test sets
testset_final_results = []

for name, model in models.items():
    # Evaluate model
    model_results = model.predict(test_x)

    p_score = precision_score(test_y, model_results, average=None, labels=[0, 1, 2])
    a_score = accuracy_score(test_y, model_results)
    r_score = recall_score(test_y, model_results, average=None, labels=[0, 1, 2])
    f1_score_result = f1_score(test_y, model_results, average=None, labels=[0, 1, 2])
    cm = confusion_matrix(test_y, model_results, labels=[0, 1, 2])
    testset_final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm ))


testset_final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(testset_final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,LR,"[0.983, 0.996, 1.0]",0.992958,"[0.996, 0.996, 0.987]","[0.989, 0.996, 0.994]","[[233, 1, 0], [1, 240, 0], [3, 0, 232]]"
1,SVC,"[0.967, 1.0, 1.0]",0.988732,"[1.0, 0.992, 0.974]","[0.983, 0.996, 0.987]","[[234, 0, 0], [2, 239, 0], [6, 0, 229]]"
2,SGDC,"[0.978, 0.971, 1.0]",0.983099,"[0.97, 0.988, 0.991]","[0.974, 0.979, 0.996]","[[227, 7, 0], [3, 238, 0], [2, 0, 233]]"
3,KNN,"[0.87, 1.0, 1.0]",0.950704,"[1.0, 0.992, 0.86]","[0.93, 0.996, 0.924]","[[234, 0, 0], [2, 239, 0], [33, 0, 202]]"
4,RF,"[0.777, 1.0, 0.994]",0.904225,"[1.0, 0.992, 0.719]","[0.875, 0.996, 0.835]","[[234, 0, 0], [1, 239, 1], [66, 0, 169]]"
5,NB,"[0.887, 0.736, 0.949]",0.842254,"[0.637, 0.938, 0.949]","[0.741, 0.825, 0.949]","[[149, 73, 12], [15, 226, 0], [4, 8, 223]]"
6,DTC,"[0.66, 1.0, 0.703]",0.784507,"[0.722, 0.963, 0.664]","[0.69, 0.981, 0.683]","[[169, 0, 65], [8, 232, 1], [79, 0, 156]]"


#### 1.4. Dumped model and input scaler using pickle

According to the evaluations, there are multiple good models at the moment, therefore, the best models are LR and Ridge.

In [18]:
with open("./model/all_sklearn.pkl", "wb") as f:
    pickle.dump(models, f)

In [19]:
with open("./model/LR_model.pkl", "wb") as f:
    pickle.dump(models["LR"], f)

In [20]:
with open("./model/SVC_model.pkl", "wb") as f:
    pickle.dump(models["SVC"], f)

In [21]:
# Dump input scaler
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)